In [1]:
import vtk
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy.spatial import KDTree
from scipy.interpolate import RegularGridInterpolator
from scipy.signal import savgol_filter
import pygmt
import importlib
import pandas as pd
import h5py
from scipy.special import erf
import matplotlib.colors as mcolors
from scipy.interpolate import interp1d
import matplotlib.patches as patches

os.chdir("../")
import parallel_curves
os.chdir("scripts_for_figures")

In [2]:
def log_RMS_misfit(naif_data, pylith_data):
    N = len(pylith_data)
    log_difference = (np.log10(pylith_data) - np.log10(naif_data))**2
    log_rms_error = np.sqrt(1/N * np.sum(log_difference))
    return log_rms_error

def log_absolute_misfit(naif_data, pylith_data):
    N = len(pylith_data)
    log_difference = abs(np.log10(pylith_data) - np.log10(naif_data))
    log_error = 1/N * np.sum(log_difference)
    return log_error

def RMS_misfit(naif_data, pylith_data):
    N = len(pylith_data)
    difference = (pylith_data - naif_data)**2
    rms_error = np.sqrt(1/N * np.sum(difference))
    return rms_error

# Load in Naif et al., Datasets

In [6]:
porosity = pd.read_excel('../../data_files/porosity_SERPENT_outerrise.xlsx', header=None).to_numpy()
resistivity = pd.read_excel('../../data_files/resistivity_SERPENT_outerrise.xlsx', header=None).to_numpy()

trench_distance = np.flip(np.arange(0, 100.1, 0.1))
depth_resistivity = np.arange(0, 12.75, 0.05)
depth_porosity = np.arange(0, 10., 0.05)

X_POR, Z_POR = np.meshgrid(trench_distance, depth_porosity)
X_RES, Z_RES = np.meshgrid(trench_distance, depth_resistivity)

# We do not include the sediment layer in our model. Set the resistivity of the sediments 
# in the Naif dataset to NAN's so that it doesn't impact our comparison. They define the
# sediment layer as being below a resistivity of 3.5 ohm meter. This occurs at roughly a
# depth of 500 m.
# basement_resistivity = 3.5
# resistivity[resistivity <= 3.5] = np.nan
X_POR = X_POR[10:, :]
Z_POR = Z_POR[10:, :] - 0.5
porosity = porosity[10:, :]

X_POR = X_POR[:, 0:-50]
Z_POR = Z_POR[:, 0:-50]
porosity = porosity[:, 0:-50]

X_RES = X_RES[10:, :]
Z_RES = Z_RES[10:, :] - 0.5
resistivity = resistivity[10:, :]

X_RES = X_RES[:, 0:-50]
Z_RES = Z_RES[:, 0:-50]
resistivity = resistivity[:, 0:-50]

In [7]:
def depth_average_porosity(base_x, bin_sizes, X_POR, Z_POR, POROSITY):
    '''
    base_x = the initial value to start binning the data (double)
    bin_sizes = an array of doubles that specifies the width of the bins. The first bin will start
                at base_x, and the subsequent bins will be shifted by the size of the previous bins.
    X_POR  = uniform mesh that specifies the x-values of the grid
    Z_POR  = uniform mesh that specifies the z-values of the grid
    POROSITY = uniform mesh that specifies the porosity values on the grid

    Returns:
    An array (object) that contains the depth averaged porosity within each bin, and an array that
    specifies the depths of the depth averaging.
    '''
    # Create an array to store the depth averages. There must be the same
    # number of depth averaged curves as there are the number of bins.
    depth_averaged_por = np.zeros(len(bin_sizes), dtype=object)
    z_vals_for_bins = np.unique(Z_POR.flatten())
    
    for i in range(len(bin_sizes)):
        average_por_at_z_vals = np.zeros(len(z_vals_for_bins))
        for j in range(len(z_vals_for_bins)):
    
            por_indices = np.where( (X_POR.flatten() >= base_x) & (X_POR.flatten() <= base_x + bin_sizes[i]) & (Z_POR.flatten() == z_vals_for_bins[j]) )
            por_at_current_z = POROSITY.flatten()[por_indices]
            average_por_at_z_vals[j] = np.average(por_at_current_z)
    
        depth_averaged_por[i] = average_por_at_z_vals
        base_x += bin_sizes[i]

    return z_vals_for_bins, depth_averaged_por

In [8]:
base_x = 5
bin_sizes = np.array([15, 20, 20, 20, 20])
z_vals_for_bins, depth_averaged_por = depth_average_porosity(base_x, bin_sizes, X_POR, Z_POR, porosity)

In [9]:
base_dir = "/Users/danieldouglas/src/cig/Outer-Rise-Faulting-PyLith/CAM_outerrise/PUBLICATION_MODELS/"
perm_model = "hatakeyama/"

pylith_models = np.array(["step01_no_faults_no_flexure",
                          "step03_faults_no_flexure",
                          "step02_no_faults_with_flexure",
                          "step04_faults_with_flexure_2.5x", 
                          "step04_faults_with_flexure_5x",
                          "step04_faults_with_flexure_10x",
                          "step04_faults_with_flexure_25x",
                          "step04_faults_with_flexure_100x"])

# pylith_models = np.array(["step05_rupture_faults_no_perm_faults_low_tol"])

porosity_file = np.loadtxt(fname=base_dir + "data_files/ave_NAIF_porosity.txt")
depth_averages_for_models_ha = np.zeros(len(pylith_models), dtype=object)

for i in range(len(pylith_models)):
    data_filename  = pylith_models[i] + "-subducting.h5"
    data_file_path = base_dir + perm_model + pylith_models[i] + '/' + data_filename
    
    with h5py.File(data_file_path, 'r') as f:
        total_porosity = f["vertex_fields"]["porosity"][:]
        total_water_content = f["vertex_fields"]["water_content"][:]
        final_porosity = total_porosity[-1]
        final_water_content = total_water_content[-1]
        
        vertices = f["geometry/vertices"]
        x_vals = vertices[:, 0]
        y_vals = vertices[:, 1]
    
        pylith_porosity = np.zeros(len(y_vals))
        for j in range(len(pylith_porosity)):
            depth_index = np.abs(y_vals[j] + porosity_file[:, 0] * 1e3).argmin()
            pylith_porosity[j] = porosity_file[:, 1][depth_index] + final_water_content[j][0]

        outerrise_x_vals   = abs(x_vals[np.where( (x_vals <= 0) & (y_vals >= -9.5e3) )] / 1e3)
        outerrise_y_vals   = abs(y_vals[np.where( (x_vals <= 0) & (y_vals >= -9.5e3) )] / 1e3)
        
        outerrise_porosity   = final_porosity[np.where( (x_vals <= 0) & (y_vals >= -9.5e3) )]
        # outerrise_porosity   = pylith_porosity[np.where( (x_vals <= 0) & (y_vals >= -9.5e3) )]

        pylith_naif_porosity = np.zeros(len(Z_POR.flatten()))
        naif_KDTREE = KDTree(np.c_[outerrise_x_vals, outerrise_y_vals])
        
        for j in range(len(pylith_naif_porosity)):
            dd, ii = naif_KDTREE.query([X_POR.flatten()[j], Z_POR.flatten()[j]])
            pylith_naif_porosity[j] = outerrise_porosity.flatten()[ii]

    z_vals_for_bins, depth_averages_for_models_ha[i] = depth_average_porosity(base_x, bin_sizes, X_POR, Z_POR, pylith_naif_porosity)

    print("Finished with model: " + str(i + 1) + " of " + str(len(pylith_models)))

binned_errors_ha = np.zeros(len(depth_averages_for_models_ha), dtype=object)
for i in range(len(binned_errors_ha)):
    binned_errors_ha[i] = np.zeros(len(depth_averages_for_models_ha[i]) - 1)
    for j in range(len(binned_errors_ha[i])):
        binned_errors_ha[i][j] = np.mean(np.abs(depth_averages_for_models_ha[i][j] - depth_averaged_por[j]) / depth_averaged_por[j])

Finished with model: 1 of 8
Finished with model: 2 of 8
Finished with model: 3 of 8
Finished with model: 4 of 8
Finished with model: 5 of 8
Finished with model: 6 of 8
Finished with model: 7 of 8
Finished with model: 8 of 8


In [10]:
base_dir = "/Users/danieldouglas/src/cig/Outer-Rise-Faulting-PyLith/CAM_outerrise/PUBLICATION_MODELS/"
perm_model = "kuang_jiao/"

pylith_models = np.array(["step01_no_faults_no_flexure",
                          "step03_faults_no_flexure",
                          "step02_no_faults_with_flexure",
                          "step04_faults_with_flexure_2.5x", 
                          "step04_faults_with_flexure_5x",
                          "step04_faults_with_flexure_10x",
                          "step04_faults_with_flexure_25x",
                          "step04_faults_with_flexure_100x"]

porosity_file = np.loadtxt(fname="/Users/danieldouglas/src/cig/Outer-Rise-Faulting-PyLith/CAM_outerrise/PUBLICATION_MODELS/data_files/NAIF_porosity.txt")
depth_averages_for_models_kj = np.zeros(len(pylith_models), dtype=object)

for i in range(len(pylith_models)):
    data_filename  = pylith_models[i] + "-subducting.h5"
    data_file_path = base_dir + perm_model + pylith_models[i] + '/' + data_filename
    
    with h5py.File(data_file_path, 'r') as f:
        total_porosity = f["vertex_fields"]["porosity"][:]
        total_water_content = f["vertex_fields"]["water_content"][:]
        final_porosity = total_porosity[-1]
        final_water_content = total_water_content[-1]
        # final_water_content[final_water_content < -0.05] = 0
        
        vertices = f["geometry/vertices"]
        x_vals = vertices[:, 0]
        y_vals = vertices[:, 1]
    
        pylith_porosity = np.zeros(len(y_vals))
        for j in range(len(pylith_porosity)):
            depth_index = np.abs(y_vals[j] + porosity_file[:, 0] * 1e3).argmin()
            pylith_porosity[j] = porosity_file[:, 1][depth_index] + final_water_content[j][0]

        outerrise_x_vals   = abs(x_vals[np.where( (x_vals <= 0) & (y_vals >= -9.5e3) )] / 1e3)
        outerrise_y_vals   = abs(y_vals[np.where( (x_vals <= 0) & (y_vals >= -9.5e3) )] / 1e3)
        
        outerrise_porosity   = final_porosity[np.where( (x_vals <= 0) & (y_vals >= -9.5e3) )]
        # outerrise_porosity   = pylith_porosity[np.where( (x_vals <= 0) & (y_vals >= -9.5e3) )]

        pylith_naif_porosity = np.zeros(len(Z_POR.flatten()))
        naif_KDTREE = KDTree(np.c_[outerrise_x_vals, outerrise_y_vals])
        
        for j in range(len(pylith_naif_porosity)):
            dd, ii = naif_KDTREE.query([X_POR.flatten()[j], Z_POR.flatten()[j]])
            pylith_naif_porosity[j] = outerrise_porosity.flatten()[ii]

    z_vals_for_bins, depth_averages_for_models_kj[i] = depth_average_porosity(base_x, bin_sizes, X_POR, Z_POR, pylith_naif_porosity)

    print("Finished with model: " + str(i + 1) + " of " + str(len(pylith_models)))

binned_errors_kj = np.zeros(len(depth_averages_for_models_kj), dtype=object)
for i in range(len(depth_averages_for_models_kj)):
    binned_errors_kj[i] = np.zeros(len(depth_averages_for_models_kj[i]) - 1)
    for j in range(len(binned_errors_kj[i])):
        binned_errors_kj[i][j] = np.mean(np.abs(depth_averages_for_models_kj[i][j] - depth_averaged_por[j]) / depth_averaged_por[j])

Finished with model: 1 of 8
Finished with model: 2 of 8
Finished with model: 3 of 8
Finished with model: 4 of 8
Finished with model: 5 of 8
Finished with model: 6 of 8
Finished with model: 7 of 8
Finished with model: 8 of 8


In [11]:
no_fault_index = 2
fault_index    = 7
ha_no_faults = np.zeros(len(depth_averages_for_models_ha[no_fault_index]) - 1)
ha_faults = np.zeros(len(depth_averages_for_models_ha[fault_index]) - 1)
kj_no_faults = np.zeros(len(depth_averages_for_models_kj[no_fault_index]) - 1)

for j in range(len(ha_no_faults)):
    ha_no_faults[j] = log_RMS_misfit(depth_averaged_por[j], depth_averages_for_models_ha[no_fault_index][j])
    ha_faults[j]    = log_RMS_misfit(depth_averaged_por[j], depth_averages_for_models_ha[fault_index][j])
    kj_no_faults[j] = log_RMS_misfit(depth_averaged_por[j], depth_averages_for_models_kj[no_fault_index][j])

ha_no_faults = ha_no_faults / np.sum(ha_no_faults) * 100
ha_faults    = ha_faults / np.sum(ha_faults) * 100
kj_no_faults = kj_no_faults / np.sum(kj_no_faults) * 100

In [13]:
panel_width  = 12.0
panel_height = 6.0

panel_dim = [0, 0, panel_width, panel_height]

x_vals = np.array([10, 30, 50, 70])

fig = plt.figure(dpi=100, figsize=(panel_width, panel_height))
main_ax = plt.axes(panel_dim)

for spine in main_ax.spines.values():
    spine.set_linewidth(15)

main_ax.set_xlabel("Distance From The Trench - km", fontsize=280)
main_ax.set_ylabel("% of Misfit", fontsize=280)

main_ax.set_xlim(8, 72)
main_ax.set_ylim(9, 41)

main_ax.set_xticks([10, 30, 50, 70])
main_ax.set_yticks([10, 25, 40])

main_ax.tick_params(axis="both", length=150, width=20, labelsize=200)

main_ax.invert_xaxis() 

main_ax.plot(x_vals, ha_no_faults, lw=60, color="black")
main_ax.plot(x_vals, ha_no_faults, lw=30, color="goldenrod", marker='o', 
             markersize=350, markerfacecolor="goldenrod", markeredgecolor="black", markeredgewidth=20,
             label="Model 4 - No Faults")

main_ax.plot(x_vals, ha_faults, lw=60, color="black")
main_ax.plot(x_vals, ha_faults, lw=30, color="khaki", marker='^', 
             markersize=300, markerfacecolor="khaki", markeredgecolor="black", markeredgewidth=20, 
             label="Model 4 - 100x Faults")

main_ax.plot(x_vals, kj_no_faults, lw=60, color="black")
main_ax.plot(x_vals, kj_no_faults, lw=30, color="plum", marker='*', 
             markersize=300, markerfacecolor="plum", markeredgecolor="black", markeredgewidth=20,
             label="Model 1 - No Faults")

main_ax.legend(loc="upper left", fontsize=250, ncols=2)

main_ax.text(78.5, 28, ".", fontsize=0.1, color="white")

main_ax.text(76.5, 45, "b)", fontsize=300)

# plt.savefig("Figures/trench_dist_error_plot.png", bbox_inches="tight")
plt.show()